In [ ]:
def run_expanding_window_backtest(data, initial_train_size, trade_lag, feature_extractor=None, device='cuda'):
    """
    Run expanding window backtest
    
    Args:
        data: DataFrame containing features and target
        initial_train_size: Initial training window size
        trade_lag: Trading lag for signals
        feature_extractor: Optional feature extractor
        device: Computing device
    """
    all_predictions = []
    expanding_metrics = []
    
    # Get total length respecting trade_lag
    total_length = len(data) - trade_lag
    
    # Start after initial training period
    for t in range(initial_train_size, total_length):
        print(f"\nWindow End: {data.index[t]}")
        
        # Get training data up to time t
        train_x = data[x_label_changes][:t]
        train_y = data['y'][:t]
        
        # Convert to tensors
        train_x = torch.tensor(train_x.values, dtype=torch.float64, device=device)
        train_y = torch.tensor(train_y.values, dtype=torch.float64, device=device)
        
        # Initialize and train model
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        model = GPRegressionModel(train_x, train_y, likelihood, feature_extractor)
        model = model.to(device)
        likelihood = likelihood.to(device)
        
        # Train model
        model, likelihood, history = train_gpr_model(
            model=model,
            likelihood=likelihood,
            train_x=train_x,
            train_y=train_y,
            device=device,
            n_epochs=100  # Reduced epochs for backtest
        )
        
        # Make prediction for next point
        with torch.no_grad():
            test_x = data[x_label_changes].iloc[t:t+1]
            test_x = torch.tensor(test_x.values, dtype=torch.float64, device=device)
            pred = model(test_x)
            pred_mean = pred.mean.cpu().numpy()[0]
        
        # Store prediction
        all_predictions.append({
            'date': data.index[t + trade_lag],
            'prediction': pred_mean,
            'actual': data['y'].iloc[t + trade_lag],
            'train_loss': history['loss'][-1]
        })
        
        # Calculate expanding window metrics
        if len(all_predictions) > 1:
            preds_df = pd.DataFrame(all_predictions)
            preds_df['returns'] = preds_df['prediction'] * preds_df['actual']
            
            # Calculate metrics
            sharpe = np.sqrt(252) * preds_df['returns'].mean() / preds_df['returns'].std()
            cum_return = (1 + preds_df['returns']).cumprod().iloc[-1] - 1
            
            expanding_metrics.append({
                'date': data.index[t + trade_lag],
                'sharpe': sharpe,
                'cum_return': cum_return,
                'window_size': t
            })
    
    # Convert results to DataFrames
    predictions_df = pd.DataFrame(all_predictions)
    metrics_df = pd.DataFrame(expanding_metrics)
    
    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(15, 15))
    
    # Plot predictions vs actual
    axes[0].plot(predictions_df['date'], predictions_df['prediction'], label='Predicted')
    axes[0].plot(predictions_df['date'], predictions_df['actual'], label='Actual')
    axes[0].set_title('Predictions vs Actual')
    axes[0].legend()
    axes[0].grid(True)
    
    # Plot cumulative returns
    predictions_df['cum_returns'] = (1 + predictions_df['returns']).cumprod()
    axes[1].plot(predictions_df['date'], predictions_df['cum_returns'])
    axes[1].set_title('Cumulative Returns')
    axes[1].grid(True)
    
    # Plot expanding window metrics
    axes[2].plot(metrics_df['date'], metrics_df['sharpe'], label='Rolling Sharpe')
    axes[2].set_title('Expanding Window Sharpe Ratio')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return predictions_df, metrics_df

# Usage:
"""
initial_train_size = 252  # One year
trade_lag = 21  # One month

predictions, metrics = run_expanding_window_backtest(
    data=data,
    initial_train_size=initial_train_size,
    trade_lag=trade_lag,
    feature_extractor=feature_extractor,
    device=device
)

# Print final metrics
print("\nFinal Backtest Metrics:")
print(f"Sharpe Ratio: {metrics['sharpe'].iloc[-1]:.2f}")
print(f"Total Return: {(metrics['cum_return'].iloc[-1]*100):.2f}%")
"""